In [ ]:
import invalid_sample_detection    as invalid
import matplotlib.pyplot           as plt
import numpy                       as np
import parameters
import load_annotations
import wfdb

%matplotlib inline
%config IPCompleter.greedy=True;

In [4]:
data_path = 'sample_data/challenge_training_data/'
ann_path = 'sample_data/challenge_training_ann/'
sample_name = 'a170s'

invalids = invalid.calculate_invalids_standard(data_path + sample_name)
print invalid.calculate_cval(invalids)

{'II': 0.9386666666666666, 'PLETH': 0.8133333333333334, 'ABP': 1.0, 'V': 0.8346666666666667}


In [ ]:
fs = parameters.FS
ann_type = 'jqrs'
start = 0
end = 5000
channel = 0

# sig, fields = wfdb.rdsamp(data_path + sample_name)
# t = np.linspace(float(start)/fs,float(end)/fs,end-start)
# plt.figure(figsize=[16,10])
# plt.plot(t, sig[start:end,channel], '-',
#          color=col[1],linewidth=2,
#          label=fields['signame'][channel])
# plt.show()

rr_intervals = load_annotations.calculate_rr_intervals_standard(ann_path + sample_name, channel, ann_type, start, end)
print sum(rr_intervals)/ len(rr_intervals)